# Quadratic Unconstrained Binary Optimization (QUBO)
## Introduction
The Quadratic Unconstrained Binary Optimization (QUBO) problem is fundamental in various fields, serving as a basis for formulating discrete combinatorial optimization problems. As an NP-hard problem, QUBO finds numerous applications across diverse domains, including machine learning, operations research, finance, chemistry, medicine, and beyond. In the realm of machine learning, its optimal performance is observed in support vector machines (SVM) and clustering algorithms.

Generically, a QUBO is defined by linear and quadratic terms, but since $x^2 = x$ when $x \in \{0,1\}$, we can simplify the optimization expression as such $f(x) = \sum_{i} \sum_{j} J_{ij} x_i x_j$, where $f: \mathbb{B}^n \rightarrow \mathbb{R}$. Note that the coefficients naturally encodes as a square symmetric matrix, so that $f(x) = x^T Q x$, where $Q$ has entries $Q_{ij}$. The goal of the optimization problem is to find the binary vector, $x^{*}$, that minimizes $f(x)$, 
$x^{*} = \mathop{\arg \min}_{x} x^T Q x$. 


## Package Installation and Remote Connection
Begin by importing the necessary packages:
- `numpy`
- `qci_client`
- `os`

In [31]:
import numpy as np
from qci_client import QciClient
import os

Establish connection with `qci_client` and QCi's server using your unique token ID and our default API URL.

In [32]:
token = "22f1f3d275d713f18f46c8020eab9848"
api_url = "https://api.qci-prod.com"
qclient = QciClient(api_token=token, url=api_url)

## Uploading a QUBO job
### Data format
To upload a square symmetric matrix or QUBO, we encode it in a sparse matrix format as shown below. We use Numpy array notation. 


In [33]:
Q = np.array([[0, -1.5, 0.5], [-1.5, 0, 0], [0.5, 0, 0]]) 

Enter your file credentials, including the QUBO file.

In [34]:
qubo_data = {
    'file_name': "smallest_objective.json",
    'file_config': {'qubo':{"data": Q}}
}


### Uploading
To upload the matrix encoded above in `qubo_data`, we use the the `qci_client` imported previously. The following line 


In [35]:
response_json = qclient.upload_file(qubo_data)

The response contains a file_id for the uploaded file. This id is provided when a job is run, along a few other parameters (see #Running). Note: the same `file_id` can be used multiple times to run a problem repeatedly. This enables an "upload once, run many times" scheme, which is especially useful for job types in which parameter searches may be involved.
Triggering a job requires two items: first a job body that contains essential and optional metadata for the job, and second, the type of job a user wants to run. 

## Running a QUBO job
### Job body
This section defines the job body for a QUBO job. Be sure to state the Dirac device of your choice under `sampler_type` and the number of samples `nsamples` required for your job.  



In [36]:
job_body = qclient.build_job_body(job_type="sample-qubo",
                                  qubo_file_id=response_json['file_id'],
                                  job_params={"sampler_type": "dirac-1", "nsamples": 5})

job_body

{'job_submission': {'problem_config': {'quadratic_unconstrained_binary_optimization': {'job_name': 'job_0',
    'job_tags': [],
    'qubo_file_id': '6602fbf138d25ec78cae8c8a',
    'num_samples': 5}},
  'device_config': {'dirac-1': {}}}}

Once the `job_body` is complete, use `job_response` to start running your job on the Dirac device.

In [37]:
job_response = qclient.process_job(job_body=job_body, job_type="sample-qubo")
job_response

Dirac allocation balance = 0
Job submitted job_id='6602fbf2832ac38bab8fe305'-: 2024/03/26 12:46:42
running: 2024/03/26 12:46:44
completed: 2024/03/26 13:04:46
Dirac allocation balance = 0


{'job_info': {'job_id': '6602fbf2832ac38bab8fe305',
  'job_submission': {'problem_config': {'quadratic_unconstrained_binary_optimization': {'qubo_file_id': '6602fbf138d25ec78cae8c8a'}},
   'device_config': {'dirac-1': {'num_samples': 1}}},
  'job_status': {'submitted_at_rfc3339nano': '2024-03-26T16:46:42.835Z',
   'queued_at_rfc3339nano': '2024-03-26T16:46:42.835Z',
   'running_at_rfc3339nano': '2024-03-26T16:46:43.695Z',
   'completed_at_rfc3339nano': '2024-03-26T16:55:46.93Z'},
  'job_result': {'file_id': '6602fe1238d25ec78cae8c94', 'device_usage_s': 2},
  'details': {'status': 'completed'}},
 'results': {'file_id': '6602fe1238d25ec78cae8c94',
  'num_parts': 1,
  'num_bytes': 235,
  'file_config': {'quadratic_unconstrained_binary_optimization_results': {'counts': [1],
    'energies': [-3],
    'solutions': [[1, 1, 0]]}}}}


Below we show how to query the result object if an error occurs:
    



In [38]:
def error_status(job_response):
    try:
        if job_response['job_info']['details']['status'] == "ERROR":
            return job_response['job_info']['details']['status'], job_response['job_info']['results']['error']
        else:
            return "No errors detected"
    except KeyError:
        return "Error: Unable to retrieve error status information from the job response"

error_status(job_response)

'No errors detected'

## Dissecting results
Under `file_config`, the job title and your results should be present in `solutions`.

### `process_job` status
Here, we have the `process_job` presenting the timestamp of each step of the running process.

`````Dirac allocation balance = 0`````

`````Job submitted job_id='xxxxx'-: yyyy/mm/dd hh:mm:ss`````

`````queued: yyyy/mm/dd hh:mm:ss`````

`````running: yyyy/mm/dd hh:mm:ss`````

`````completed: yyyy/mm/dd hh:mm:ss`````

`````Dirac allocation balance = 0`````

### `job_response` status
Details pertaining to the configuration of your job are presented here, including information about the job submitted, `job_info`, the device chosen, `device_config`, the status of the job repeated from `process_job`, `job_status`, and details of whether the job was completed or uncompleted, `details`.

```json
{
  "job_info": {
    "job_id": "6601958b832ac38bab8fe27f",
    "job_submission": {
      "problem_config": {
        "quadratic_unconstrained_binary_optimization": {
          "qubo_file_id": "6601953438d25ec78cae8a65"
        }
      },

      "device_config": {
        "dirac-1": {
          "num_samples": 1
        }
      }
    },
    
    "job_status": {
      "submitted_at_rfc3339nano": "yyyy-mm-ddThh:mm:ss",
      "queued_at_rfc3339nano": "yyyy-mm-ddThh:mm:ss",
      "running_at_rfc3339nano": "yyyy-mm-ddThh:mm:ss",
      "completed_at_rfc3339nano": "yyyy-mm-ddThh:mm:ss"
    },

    "job_result": {
      "file_id": "660198fc38d25ec78cae8a67",
      "device_usage_s": 1
    },
    
    "details": {
      "status": "completed"
    }
  },
}

### `job_response` results
Within the `job_response` we can identify the `solutions`, `energies`, and `counts` resulting from the job.

`Solutions` are the binary solutions of the polynomial function.
`Energies` are values obtained by evaluating a polynomial function at each solution obtained from the optimization process.
`Counts` are the number of times a solution was found per iteration.

```json
{
  "results": {
    "file_id": "660198fc38d25ec78cae8a67",
    "num_parts": 1,
    "num_bytes": 235,
    "file_config": {
      "quadratic_unconstrained_binary_optimization_results": {
        "counts": [1],
        "energies": [-3],
        "solutions": [[1, 1, 0]]
      }
    }
  }
}